In [1]:
from langchain_chroma import Chroma
import os
from pyprojroot import here
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from pprint import pprint

**Load environment variables and configs**

In [ ]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

VECTORDB_DIR = "data/airline_policy_vectordb"
K=2

In [2]:
import os
import getpass
from dotenv import load_dotenv
from pyprojroot import here
from typing import List
from pprint import pprint

from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain.document_loaders import PyMuPDFLoader  # 例如加载PDF文件
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# 强制覆盖已存在的环境变量load_dotenv(override=True)

# os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
print(os.environ["OPENAI_API_KEY"],OPENAI_API_KEY)

sk-rKcLAkdmwzWdMdOTdlXFKZuLEcY8drtZJ2DsXZZkTC9sQoE0 sk-rKcLAkdmwzWdMdOTdlXFKZuLEcY8drtZJ2DsXZZkTC9sQoE0


In [ ]:
# 如果没有设置 GROQ_API_KEY，则提示用户输入
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")
#  初始化 Llama 模型，使用 Groq 后端
client = init_chat_model("llama-3.3-70b-specdec", model_provider="groq", temperature=0)
print(client)


In [4]:
# 加载文档（此处以PDF为例）
# sqldb_directory = here("data/env-work.pdf")
# loader = PyMuPDFLoader(sqldb_directory)

# from openai import OpenAI
# client = OpenAI(
#     base_url="https://xiaoai.plus/v1",
#     api_key=OPENAI_API_KEY
# )

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-small"
# )

# print(response.data[0].embedding)

# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI

# 设置 OpenAI API 密钥
import os

# 创建 OpenAIEmbeddings 实例
embeddings = OpenAIEmbeddings(
    base_url="https://xiaoai.plus/v1",
    api_key=OPENAI_API_KEY
)

# 输入文本
text = "Your text string goes here"

# 获取文本的嵌入
embedding = embeddings.embed_documents(text)

# 输出嵌入
print(embedding[0])


[-0.01936718448996544, -0.03900187090039253, 0.017173664644360542, -0.019688189029693604, -0.02345997467637062, 0.029050776734948158, -0.02317909710109234, -0.024784112349152565, -0.01214461587369442, -0.013214626349508762, 0.00809863954782486, 0.019139807671308517, 0.012773247435688972, 0.0038921621162444353, -0.03509633615612984, -0.014739390462636948, 0.02633562684059143, 0.014257886447012424, 0.04322841390967369, -0.023794353008270264, -0.010191847570240498, 0.009335839189589024, -0.011723299510776997, -0.018524551764130592, -0.011455796658992767, 0.0026599785778671503, 0.006219434551894665, 0.0004802506882697344, 0.017494667321443558, 0.0005730406264774501, 0.03196655586361885, -0.009135212749242783, -0.0019360497826710343, -0.015087144449353218, -0.010486099869012833, -0.01659853383898735, -0.0063331229612231255, -0.025974499061703682, -0.012672933749854565, -0.004938765894621611, 0.017748795449733734, 0.01911305822432041, -0.004945453722029924, -0.00856008194386959, -0.011910551

**Load the vectorDB**

In [ ]:
vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=OpenAIEmbeddings()
)
print("Number of vectors in vectordb:",
      vectordb._collection.count(), "\n\n")

**Sample Query**

In [ ]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"

**Perform the vector Search**

In [ ]:
docs = vectordb.similarity_search(message, k=K)

In [ ]:
docs

**Prepare the prompt for the GPT model**

In [ ]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in docs:
    retrieved_content += f"{doc.page_content}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

Prepared prompt

In [ ]:
pprint(prompt)

**Pass the prompt to the GPT model and get the response**

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    # tream=True,
    temperature=0)

messages=[
    {"role": "system", "content": "You will receive a user's query and possible content where the answer might be. If the answer is found, provide it, if not, state that the answer does not exist."},
    {"role": "user", "content": prompt}
]

# Call the model with the messages
response = chat.invoke(messages)

# Print the response
print(response)

Printing the response

In [ ]:
pprint(response.content)

**RAG Tool design using LangChain**

In [ ]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str)->str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=OpenAIEmbeddings()
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

In [ ]:
pprint(lookup_swiss_airline_policy.invoke("can I cancel my ticket?"))